In [7]:

import pandas as pd
import matplotlib.pyplot as plt
import neurokit2 as nk
import os
import re
import numpy as np

In [8]:
def find_data_folder(folder_path, start_path='/content/drive'):
  '''
    :file_path: The complete path of the folder that includes the files.
    :start_path: The root folder on Drive from which we start the search.
  '''
  #Search the root folder of the data and if found, continue searching the full path.
  start_data_folder = folder_path.split('/')[0]
  for root, dirs, files in os.walk(start_path):
      if start_data_folder in dirs:
          return os.path.join(root, folder_path)
  raise Exception(f"{start_data_folder} folder not found. Please check the directory structure.")

def read_store_files(files_path, file_type='.csv'):
  '''
    :files_path: The complete Drive path that includes the files
  '''

  data_list = []
  files_names = []
  for file in os.listdir(files_path):
    if file.endswith(file_type):
      file_path = os.path.join(files_path, file)
      if file_type == '.csv':
        data = pd.read_csv(file_path)
      elif file_type == '.xlsx':
        data = pd.read_excel(file_path)

      data_list.append(data)
      files_names.append(file)

  return data_list, files_names

def zero_pad_filenames(annotation_files_names):
    # Create a list to hold the zero-padded filenames
    padded_filenames = []

    for filename in annotation_files_names:
        # Extract the number part from the filename
        parts = re.split(r'(\d+)', filename)
        # Zero-pad the number part
        parts[1] = parts[1].zfill(2)
        # Reconstruct the filename
        padded_filename = ''.join(parts)
        padded_filenames.append(padded_filename)

    return padded_filenames

def sort_data(files_names, data):
    # Get the zero-padded filenames
    padded_filenames = zero_pad_filenames(files_names)
    
    # Create a list of tuples (padded_filename, original_filename, annotation_data)
    combined_list = list(zip(padded_filenames, files_names, data))

    # Sort the combined list based on the zero-padded filenames
    combined_list.sort()

    # Unzip the sorted list back into the separate components
    sorted_filenames = [original_filename for _, original_filename, _ in combined_list]
    sorted_data = [data for _, _, data in combined_list]

    return sorted_filenames, sorted_data

In [9]:
# Retrieve folders
annotations_folder = 'CASE_full/data/interpolated/annotations'
physiological_folder = 'CASE_full/data/interpolated/physiological'
metadata_folder = 'CASE_full/metadata'

print(f'Path of Annotation files: {annotations_folder}')
print(f'Path of Physiological files: {physiological_folder}')
print(f'Path of metadata files: {metadata_folder}')

# Retrieve data files
annotation_data, annotation_files_names = read_store_files(annotations_folder)
physio_data, physio_files_names = read_store_files(physiological_folder)

annotation_files_names, annotation_data = sort_data(annotation_files_names, annotation_data)
physio_files_names, physio_data = sort_data(physio_files_names, physio_data)

print(f'Number of Annotation files: {len(annotation_data)}\nNames: {annotation_files_names}')
print(f'Number of Physiological files: {len(physio_data)}\nNames: {physio_files_names}')

# Retrieve metadata
metadata, metadata_names = read_store_files(metadata_folder,'.xlsx')
print(f'Metadata names: {metadata_names}')

# Strip column names
for i in range(len(metadata)):
  metadata[i].columns = metadata[i].columns.str.strip()

for i, file in enumerate(metadata_names):
  if file == 'videos.xlsx':
    videos_data = metadata[i].drop(metadata[i].columns[2:], axis=1).drop([0]).rename(columns={'Video-label': 'label', 'Video-ID': 'video_id'}).dropna()
    videos_data['video_id'] = videos_data['video_id'].astype(int)
  elif file == 'participants.xlsx':
    participant_data = metadata[i].rename(columns={'Participant-ID': 'participant_id', 'Age-Group': 'age_group', 'Video Sequence Used': 'sequence'})

  elif file == 'videos_duration_num.xlsx':
    duration_data = metadata[i].rename(columns={'video-ID': 'video_id', 'video-duration (in ms)': 'duration'})

  elif file == 'seqs_order_num.xlsx':
    sequence_order_data = metadata[i]


del metadata

Path of Annotation files: CASE_full/data/interpolated/annotations
Path of Physiological files: CASE_full/data/interpolated/physiological
Path of metadata files: CASE_full/metadata
Number of Annotation files: 30
Names: ['sub_1.csv', 'sub_2.csv', 'sub_3.csv', 'sub_4.csv', 'sub_5.csv', 'sub_6.csv', 'sub_7.csv', 'sub_8.csv', 'sub_9.csv', 'sub_10.csv', 'sub_11.csv', 'sub_12.csv', 'sub_13.csv', 'sub_14.csv', 'sub_15.csv', 'sub_16.csv', 'sub_17.csv', 'sub_18.csv', 'sub_19.csv', 'sub_20.csv', 'sub_21.csv', 'sub_22.csv', 'sub_23.csv', 'sub_24.csv', 'sub_25.csv', 'sub_26.csv', 'sub_27.csv', 'sub_28.csv', 'sub_29.csv', 'sub_30.csv']
Number of Physiological files: 30
Names: ['sub_1.csv', 'sub_2.csv', 'sub_3.csv', 'sub_4.csv', 'sub_5.csv', 'sub_6.csv', 'sub_7.csv', 'sub_8.csv', 'sub_9.csv', 'sub_10.csv', 'sub_11.csv', 'sub_12.csv', 'sub_13.csv', 'sub_14.csv', 'sub_15.csv', 'sub_16.csv', 'sub_17.csv', 'sub_18.csv', 'sub_19.csv', 'sub_20.csv', 'sub_21.csv', 'sub_22.csv', 'sub_23.csv', 'sub_24.csv', '

In [10]:
def plot_signal_segment(raw_signal, cleaned_signal, title, start_time, duration, sampling_rate=1000):
    """ Utility. Makes two graphs of a raw and a cleaned signal for demonstration """
    start_sample = int(start_time * sampling_rate)
    end_sample = int((start_time + duration) * sampling_rate)

    plt.figure(figsize=(14, 6))

    plt.subplot(2, 1, 1)
    plt.plot(raw_signal[start_sample:end_sample], label='Raw Signal')
    plt.title(f'Raw {title} Segment ({start_time}-{start_time + duration} sec)')
    plt.xlabel('Samples')
    plt.ylabel('Amplitude')
    plt.legend()

    plt.subplot(2, 1, 2)
    plt.plot(cleaned_signal[start_sample:end_sample], label='Cleaned Signal')
    plt.title(f'Cleaned {title} Segment ({start_time}-{start_time + duration} sec)')
    plt.xlabel('Samples')
    plt.ylabel('Amplitude')
    plt.legend()

    plt.tight_layout()
    plt.show()
    
def extract_segments(cleaned_signal, start_time, end_time, original_timestamps):
    """ Utility. Extracts the signal segment given timestamps """
    start_index = original_timestamps[original_timestamps >= start_time].index[0]
    end_index = original_timestamps[original_timestamps <= end_time].index[-1]

    segment = cleaned_signal[start_index:end_index+1]
    return segment

def preprocess_signal(signal, data_type, visualization=False):
    """ Just cleans the signal (NOTE: ok for now).
    Set visualization to True to see a before-after example
    """
    if data_type == "ecg":
        cleaned_signal = nk.ecg_clean(signal, sampling_rate=1000)
    elif data_type == "gsr":
        cleaned_signal = nk.eda_clean(signal, sampling_rate=1000)
    else:
        raise ValueError("Unsupported data type")

    if visualization:
        # Plot a 5-second segment for demonstration
        plot_signal_segment(signal, cleaned_signal, 'Signal', start_time=400, duration=5 if data_type=='ecg' else 200)
    return cleaned_signal

In [13]:
def extract_features(segment, data_type, sampling_rate=1000):
    """ Extracts ECG and GSR features.

    ECG:
    - Heart Rate: Mean heart rate over the segment.
    - HRV SDNN: Standard deviation of NN intervals (HRV time-domain).
    - Mean RR Interval: Mean of RR intervals.
    - Poincaré SD1: Standard deviation of the points perpendicular to the line of identity in the Poincaré plot (HRV non-linear).
    - LF/HF Ratio: Ratio of low-frequency to high-frequency power (HRV frequency-domain).
    - RSA: Respiratory sinus arrhythmia, approximated by RMSSD.
    - PSD Total Power: Total power of the power spectral density.
    - PSD LF Power: Power in the low-frequency band of the PSD.
    - PSD HF Power: Power in the high-frequency band of the PSD.

    GSR:
    # TODO:
    - SCL: Skin conductance level (mean tonic level).
    - SCR: Skin conductance response (mean phasic level).
    - Peak Amplitude: Maximum amplitude of the SCR peaks.
    - PSD Total Power: Total power of the power spectral density.
    - PSD LF Power: Power in the low-frequency band of the PSD.
    - PSD HF Power: Power in the high-frequency band of the PSD.

    Parameters:
    - segment: Array-like, the segment of the signal to analyze.
    - data_type: str, type of the signal ("ecg" or "gsr").
    - sampling_rate: int, sampling rate of the signal in Hz.

    Returns:
    - dict: A dictionary containing the extracted features.
    """
    if data_type == "ecg":
        signals, info = nk.ecg_process(segment, sampling_rate=sampling_rate)
        heart_rate = signals['ECG_Rate']

        peaks, info = nk.ecg_peaks(segment, sampling_rate=sampling_rate)
        hrv_time = nk.hrv_time(peaks, sampling_rate=sampling_rate).iloc[0].to_dict()
        hrv_nonlinear = nk.hrv_nonlinear(peaks, sampling_rate=sampling_rate).iloc[0].to_dict()
        hrv_frequency = nk.hrv_frequency(peaks, sampling_rate=sampling_rate).iloc[0].to_dict()

        psd = nk.signal_psd(segment, sampling_rate=sampling_rate)

        features = {
            "Heart Rate": heart_rate.mean(),
            "HRV SDNN": hrv_time['HRV_SDNN'],
            "Mean RR Interval": hrv_time['HRV_MeanNN'],
            "Poincaré SD1": hrv_nonlinear['HRV_SD1'],
            "LF/HF Ratio": hrv_frequency['HRV_LFHF'],
            "RSA": hrv_time['HRV_RMSSD'],
            "PSD Total Power": psd['Power'].sum(),
            "PSD LF Power": psd['Power'][psd['Frequency'] <= 0.15].sum(),
            "PSD HF Power": psd['Power'][(psd['Frequency'] > 0.15) & (psd['Frequency'] <= 0.4)].sum()
        }
    elif data_type == "gsr":
        signals, info = nk.eda_process(segment, sampling_rate=sampling_rate)
        scl = signals['EDA_Tonic'].mean()
        scr = signals['EDA_Phasic'].mean()

        peaks, info = nk.eda_peaks(segment, sampling_rate=sampling_rate)
        peak_amplitude = signals['EDA_Phasic'].max()
        # this is not useful now, cause it's always one
        # peak_frequency = len(peaks['SCR_Peaks']) / len(segment)

        psd = nk.signal_psd(segment, sampling_rate=sampling_rate)

        features = {
            "SCL": scl,
            "SCR": scr,
            "Peak Amplitude": peak_amplitude,
            "PSD Total Power": psd['Power'].sum(),
            "PSD LF Power": psd['Power'][psd['Frequency'] <= 0.15].sum(),
            "PSD HF Power": psd['Power'][(psd['Frequency'] > 0.15) & (psd['Frequency'] <= 0.4)].sum()
        }
    else:
        raise ValueError("Unsupported data type")
    return features

def get_segment_timestamps(data):
    """ Gets the df data based on the timestamps
    (NOTE: this allows for further segmentation by being flexible and
    not relying on adding new df columns or indices)
    """
    # Get data only for videos 7 and 8 (scary videos)
    scary_1_data = data[data['video'] == 7]
    scary_2_data = data[data['video'] == 8]

    # Get the start and end timestamps
    scary_1_start = scary_1_data['daqtime'].iloc[0]
    scary_1_end = scary_1_data['daqtime'].iloc[-1]
    scary_2_start = scary_2_data['daqtime'].iloc[0]
    scary_2_end = scary_2_data['daqtime'].iloc[-1]

    # Swap the videos if needed (based on presented order)
    if scary_1_start > scary_2_start: # compare start timestamps
        (scary_1_start, scary_1_end), (scary_2_start, scary_2_end) = (scary_2_start, scary_2_end), (scary_1_start, scary_1_end)

    return (scary_1_start, scary_1_end), (scary_2_start, scary_2_end)



In [15]:
results = []

for participant in range(len(physio_data)):
    print(f"\n----------- Participant {participant+1} data -----------------\n")

    ecg_signal = physio_data[participant]['ecg']
    gsr_signal = physio_data[participant]['gsr']
    original_timestamps = physio_data[participant]['daqtime']

    # Clean signals (set to True to visualize before-after)
    ecg_cleaned = preprocess_signal(ecg_signal, data_type="ecg", visualization=False)
    gsr_cleaned = preprocess_signal(gsr_signal, data_type="gsr", visualization=False)

    # Get timestamps for scary videos
    (scary_1_start, scary_1_end), (scary_2_start, scary_2_end) = get_segment_timestamps(physio_data[participant])

    # Extract segment data (NOTE: something like this could be used later even if we segment the videos further)
    ecg_scary_1_segment = extract_segments(ecg_cleaned, scary_1_start, scary_1_end, original_timestamps)
    ecg_scary_2_segment = extract_segments(ecg_cleaned, scary_2_start, scary_2_end, original_timestamps)

    gsr_scary_1_segment = extract_segments(gsr_cleaned, scary_1_start, scary_1_end, original_timestamps)
    gsr_scary_2_segment = extract_segments(gsr_cleaned, scary_2_start, scary_2_end, original_timestamps)

    # Calculate ECG and GSR features for both videos
    ecg_features_scary_1 = extract_features(ecg_scary_1_segment, data_type="ecg")
    ecg_features_scary_2 = extract_features(ecg_scary_2_segment, data_type="ecg")

    gsr_features_scary_1 = extract_features(gsr_scary_1_segment, data_type="gsr")
    gsr_features_scary_2 = extract_features(gsr_scary_2_segment, data_type="gsr")

    # Calculate differences for all features (to use in statistical analysis)
    ecg_features_diff = {f'diff_ecg_{key}': ecg_features_scary_2[key] - ecg_features_scary_1[key] for key in ecg_features_scary_1.keys()}
    gsr_features_diff = {f'diff_gsr_{key}': gsr_features_scary_2[key] - gsr_features_scary_1[key] for key in gsr_features_scary_1.keys()}

    # Compile all features into a single dictionary
    participant_features = {'participant': participant}
    participant_features.update({f'scary_1_ecg_{key}': value for key, value in ecg_features_scary_1.items()})
    participant_features.update({f'scary_2_ecg_{key}': value for key, value in ecg_features_scary_2.items()})
    participant_features.update(ecg_features_diff)
    participant_features.update({f'scary_1_gsr_{key}': value for key, value in gsr_features_scary_1.items()})
    participant_features.update({f'scary_2_gsr_{key}': value for key, value in gsr_features_scary_2.items()})
    participant_features.update(gsr_features_diff)

    # Add to results
    results.append(participant_features)


----------- Participant 1 data -----------------


----------- Participant 2 data -----------------


----------- Participant 3 data -----------------


----------- Participant 4 data -----------------


----------- Participant 5 data -----------------


----------- Participant 6 data -----------------


----------- Participant 7 data -----------------


----------- Participant 8 data -----------------


----------- Participant 9 data -----------------


----------- Participant 10 data -----------------


----------- Participant 11 data -----------------


----------- Participant 12 data -----------------


----------- Participant 13 data -----------------


----------- Participant 14 data -----------------


----------- Participant 15 data -----------------


----------- Participant 16 data -----------------


----------- Participant 17 data -----------------


----------- Participant 18 data -----------------


----------- Participant 19 data -----------------


----------- Particip

In [16]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Save
output_filepath = 'ecg_gsr_features.csv'
results_df.to_csv(output_filepath, index=False)

print(f"Results saved to {output_filepath}.")

Results saved to ecg_gsr_features.csv.
